In [2]:
def ananlyze_ticker():

    from selenium import webdriver
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.support.ui import WebDriverWait
    from bs4 import BeautifulSoup
    from datetime import datetime
    from selenium.webdriver.chrome.service import Service
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.common.exceptions import TimeoutException
    from selenium.webdriver.common.by import By
    from bs4 import BeautifulSoup
    import math
    from tqdm import tqdm
    import time
    import pandas as pd

    #인베스팅 닷컴 로그인 진행하기
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    nationcode = '5'
    url = f'https://www.investing.com/stock-screener/?sp=country::{nationcode}|sector::a|industry::a|equityType::a%3Ceq_market_cap;1'
    driver.get(url)

    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="resultsTable"]/tbody')))

    #마지막 번호 추출하기
    end_num = driver.find_element(By.CLASS_NAME, value='js-total-results').text
    end_num = math.ceil(int(end_num) /49)

    html = BeautifulSoup(driver.page_source, 'lxml')

    #html 테이블에서 마지막 행 값 추출하기
    html_table = html.select('table.genTbl.openTbl.resultsStockScreenerTbl.elpTbl')
    df_table = pd.read_html(html_table[0].prettify())

    df_columns = df_table[0].columns.tolist()


    #티커 및 정보를 추출하여 데이터프레임으로 만들기
    all_data_df = []
    for i in tqdm(range(1, end_num)):

        url = f'https://www.investing.com/stock-screener/?sp=country::{nationcode}|sector::a|industry::a|equityType::a%3Ceq_market_cap;{i}'
        driver.get(url)

        try :
            WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="resultsTable"]/tbody')))

        except TimeoutException :
            try :
                driver.refresh()
                WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="resultsTable"]/tbody')))
                time.sleep(1)
            except TimeoutException :
                pass

        html = BeautifulSoup(driver.page_source, 'lxml')
        
        html_table = html.select('table.genTbl.openTbl.resultsStockScreenerTbl.elpTbl')
        df_table = pd.read_html(html_table[0].prettify())
        df_table_select = df_table[0][df_columns]

        all_data_df.append(df_table_select)

        time.sleep(2)

    all_data_df_bind = pd.concat(all_data_df, axis=0)


    #가져온 데이터 값에서 필요없는 열값을 제거하기
    all_data_df_bind.drop(columns="var columnsSettings_stock_screener = new ColumnsSettings(  'ltr', // strHtmlDir  'resultsContainer', // containerId  'resultsContainer', // tab  'stock_screener', // id  'Currently selected: X (max 8)', // message  '8', // intMaxCheckboxes  '1' // intMinOptCheckboxes \t);  Adjust table  Name  Symbol  Exchange  Sector  Industry  Last  Chg. %  Market Cap  Vol.  P/E Ratio  MACD (12,26 / 1D)  Revenue  Average Vol. (3m)  EPS  Beta  Dividend  Yield  15 Minutes  Hourly  Daily  Weekly  Monthly  Daily  1 Week  1 Month  YTD  1 Year  3 Years  1-Year Change  Dividend Yield (%)  P/E Ratio (TTM)  Price to Sales (TTM)  Price to Cash Flow (MRQ)  Price to Free Cash Flow (TTM)  Price to Book (MRQ)  Price to Tangible Book (MRQ)  EPS(MRQ) vs Qtr. 1 Yr. Ago  EPS(TTM) vs TTM 1 Yr. Ago  5 Year EPS Growth  Sales (MRQ) vs Qtr. 1 Yr. Ago  Sales (TTM) vs TTM 1 Yr. Ago (TTM)  5 Year Sales Growth  5 Year Capital Spending Growth  Asset Turnover (TTM)  Inventory Turnover (TTM)  Revenue/Employee (TTM)  Net Income/Employee (TTM)  Receivable Turnover (TTM)  52 wk Range - High  52 wk Range - Low  % Change from 52 wk High  % Change from 52 wk Low  Previous Month % Change  Gross margin (TTM)  Gross Margin (5YA)  Operating margin (TTM)  Operating margin (5YA)  Pretax margin (TTM)  Pretax margin (5YA)  Net Profit margin (TTM)  Net Profit margin (5YA)  Quick Ratio (MRQ)  Current Ratio (MRQ)  LT Debt to Equity (MRQ)  Total Debt to Equity  Dividend Yield 5 Year Avg. (5YA)  Dividend Growth Rate (ANN)  Payout Ratio (TTM)  ADX (14 / 1D)  ATR (14 / 1D)  Bull/Bear Power (13 / 1D)  CCI (14 / 1D)  Highs/Lows (14 / 1D)  ROC (1D)  RSI (14 / 1D)  STOCH (14 / 1D)  STOCHRSI (14 / 1D)  Ultimate Oscillator (14 /1D)  Williams %R (1D)  Advanced Metrics  Currently selected:  6  (max\xa08\xa0)  Apply  $('#colSelectPopup_stock_screener').click(function(e) {  e.stopPropagation(); \t});", inplace=True)
    all_data_df_bind.drop(columns="Unnamed: 0", inplace=True)



    # 끌어온 데이터 정리 및 날짜, 국가 정보값 입력하기
    import numpy as np

    data_country = html.find(class_='js-search-input inputDropDown')['value']
    all_data_df_bind['country'] = data_country
    all_data_df_bind['date'] = datetime.today().strftime('%y-%m-%d')
    all_data_df_bind = all_data_df_bind[~all_data_df_bind['Symbol'].isnull()]
    all_data_df_bind = all_data_df_bind[~all_data_df_bind['Symbol'].isnull()]
    all_data_df_bind = all_data_df_bind.drop_duplicates(['Symbol'])
    all_data_df_bind.reset_index(inplace=True, drop=True)
    all_data_df_bind = all_data_df_bind.replace({np.nan:None})

    all_data_df_bind.head()
    ticker_list = all_data_df_bind
    driver.quit()

    #SQL 서버에 데이터값 입력하기(로컬)


    #마법공식 종목을 알려주는 사이트에서 정보값 가져오기
    from selenium import webdriver
    from selenium.webdriver.common.by import By

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    url = 'https://www.magicformulainvesting.com/Account/LogOn'
    driver.get(url)

    id_input = driver.find_element(By.NAME, "Email")  
    id_input.send_keys("sidung2@naver.com")  
    pw_input = driver.find_element(By.NAME, "Password")  
    pw_input.send_keys("!!Ll575212")  
    login_input = driver.find_element(By.NAME, "login")
    login_input.click()


    driver.get('https://www.magicformulainvesting.com/Screening/StockScreening')
    select_input = driver.find_elements(By.ID, "Select30")[1]
    select_input.click()


    login_input = driver.find_element(By.NAME, "stocks")
    login_input.click()

    time.sleep(3)


    html = BeautifulSoup(driver.page_source, 'html.parser')


    from bs4 import BeautifulSoup

    table = html.find('table', {'class': ['divheight', 'screeningdata']})
    table_html = str(table)

    # 이 문자열을 pandas DataFrame으로 변환합니다.
    df = pd.read_html(table_html)[0]

    df_tickers = df['Ticker'].tolist()

    symbols = df_tickers
    filtered_ticker = ticker_list[ticker_list['Symbol'].isin(symbols)]
    filtered_ticker.head()

    sorted_value = filtered_ticker[['Name','Symbol','Exchange','Sector','Industry','Last','Chg. %','P/E Ratio','Price to Tangible Book (MRQ)','Net Profit margin (TTM)','EPS','Dividend Yield (%)','RSI (14 / 1D)','Beta','Daily']]
    sorted_value = sorted_value[(sorted_value['Exchange'] == 'NYSE') | (sorted_value['Exchange'] == 'NASDAQ')]
    sorted_value = sorted_value[(sorted_value['Daily'] == "Strong Buy") | (sorted_value['Daily'] == "Buy") | (sorted_value['Daily'] == "Neutral")]
    sorted_value = sorted_value[sorted_value['Sector'] != "Healthcare"]
    sorted_value = sorted_value[sorted_value['Sector'] != "Energy"]
    sorted_value = sorted_value[sorted_value['P/E Ratio'].astype(float) < 10 ]


    return sorted_value


sorted_value = ananlyze_ticker()





100%|██████████| 237/237 [17:22<00:00,  4.40s/it]


In [43]:
df = sorted_value.sort_values(by="Price to Tangible Book (MRQ)", ascending=True)
df = df[df['Price to Tangible Book (MRQ)'] != "-" ]
df = df[df['Price to Tangible Book (MRQ)'].astype(float) > 1 ]

import pandas as pd

df["RSI (14 / 1D)"] = pd.to_numeric(df["RSI (14 / 1D)"], errors='coerce')
df = df.sort_values(by="RSI (14 / 1D)", ascending=True)
df

,Name,Symbol,Exchange,Sector,Industry,Last,Chg. %,P/E Ratio,Price to Tangible Book (MRQ),Net Profit margin (TTM),EPS,Dividend Yield (%),RSI (14 / 1D),Beta,Daily
3181,Alpha Metallurgical Resources,AMR,NYSE,Basic Materials,Metals & Mining,147.12,-2.03%,1.76,1.45,33.46,79.12,1.36%,50.22,1.27,Neutral
3327,Warrior Met Coal,HCC,NYSE,Basic Materials,Metals & Mining,35.42,-1.34%,2.56,1.18,36.49,13.1,0.79%,50.49,1.12,Buy
4248,Perdoceo Education,PRDO,NASDAQ,Academic & Educational Services,Miscellaneous Educational Service Providers,12.42,0.32%,8.51,1.79,13.88,1.45,-,51.49,1.20,Neutral
3421,Buckle,BKE,NYSE,Consumer Cyclicals,Specialty Retailers,34.09,1.13%,6.98,4.12,18.37,4.91,4.11%,57.44,0.95,Strong Buy
2207,Mueller Industries,MLI,NYSE,Industrials,"Machinery, Tools, Heavy Vehicles, Trains & Ships",81.49,-2.41%,6.51,2.72,17.07,12.09,1.47%,58.53,1.10,Strong Buy
1605,Williams-Sonoma,WSM,NYSE,Consumer Cyclicals,Specialty Retailers,125.05,-0.41%,7.68,5.80,12.07,15.43,2.88%,59.84,-,Strong Buy
4775,Mastercraft Boat,MCFT,NASDAQ,Consumer Cyclicals,Leisure Products,30.1,1.01%,8.54,4.67,7.61,3.25,-,61.17,1.76,Strong Buy
4117,Ferroglobe,GSM,NASDAQ,Basic Materials,Metals & Mining,5.07,0.2%,2.90,2.34,13.58,1.65,-,62.19,2.22,Strong Buy
3864,Arhaus,ARHS,NASDAQ,Consumer Cyclicals,Specialty Retailers,8.63,2.25%,7.67,5.05,12.02,1.12,-,64.65,2.20,Strong Buy
4687,Resources Connection,RGP,NASDAQ,Industrials,Professional & Commercial Services,16.85,-0.18%,8.21,3.01,7.81,1.90,3.32%,64.91,0.71,Strong Buy
